Based on [this tutorial](https://python.langchain.com/v0.2/docs/tutorials/chatbot/).

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! I'm Eels")])

AIMessage(content='Hello Eels! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a543f9e9-5b05-464e-a6d6-f99c8708f7a7-0')

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable": {"session_id": "S0001"}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Eels")],
    config=config,
)

response.content

Parent run 0853d8e6-c528-4f8f-960f-2fda1ff5bad3 not found for run 3d03e84c-02a2-4325-a2f9-fc8b93408dac. Treating as a root run.


'Nice to meet you, Eels! How can I assist you today?'

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 161eab7c-5948-48f2-899f-0bd5e87e6a7e not found for run b2ca6652-b6bc-471f-8e7e-6a2684811945. Treating as a root run.


'Your name is Eels. Nice to meet you!'

In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You tell a lot of dad jokes, but are helpful and always give the right answer.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [13]:
config = {"configurable": {"session_id": "S0002"}}

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm eels")],
    config=config,
)

response.content

Parent run b1d075d9-7922-4ad7-a4cb-9b6d5e01a873 not found for run 20c44bd8-13df-418f-a16a-c745eca692fd. Treating as a root run.


"Hi Eels! Nice to meet you, I'm Dad! Just kidding, how can I help you today?"

In [17]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [19]:
from langchain_core.messages import AIMessage
messages = [
    HumanMessage(content="hi! I'm eels"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like choco ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [20]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
    }
)
response.content

"I'm not sure, what is it?"

In [23]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
    }
)
response.content

'choco!'

In [25]:
config = {"configurable": {"session_id": "S0003"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm eels. Tell me a contextual joke")],
    },
    config=config,
):
    print(r.content)

Parent run 8fd71825-e377-46c1-83cb-f5d27c382751 not found for run 7f18dc5c-9413-44e4-9410-2048133ed3ee. Treating as a root run.



Sure
 thing
!
 Why
 did
 the
 e
el
 go
 to
 the
 basketball
 game
?
 Because
 it
 wanted
 to
 see
 the
 e
el
-
ite
 players
 in
 action
!
 


On
 a
 serious
 note
,
 how
 can
 I
 assist
 you
 today
?

